In [1]:
# Importações

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn import svm
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    cohen_kappa_score,
    roc_auc_score,
    confusion_matrix
)

import pandas as pd
import numpy as np

from nltk.corpus import stopwords

import re

import itertools

# Cross validate

from sklearn.model_selection import cross_validate

# Modelos

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

/home/ABTLUS/pedro220058/.local/lib/python3.10/site-packages/nltk/metrics/association.py:26: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.6)
  from scipy.stats import fisher_exact


In [2]:
# Importação do Dataframe contendo títulos de notícias relacionadas a saúde.

dataframe = pd.read_csv('../combined_df.zip')

display(dataframe)

,title,link,date,Site,noticia_falsa,corpo_texto
0,"Ken Frazier, Merck, disse pressa vacinar liber...",https://www.boatos.org/saude/ken-frazier-merck...,30/10/2020,NaN,0,“ prometer vacinar covid gravar desserviço o o...
1,Coronavírus: Últimas notícias sabemos sexta-fe...,https://noticias.uol.com.br/saude/ultimas-noti...,08/05/2020 13h09Atualizada em 08/05/2020 22h35,NaN,1,particularidade perigoso o brasil pandemia cor...
2,menino indiano prever coronavírus antar pandemia,https://www.e-farsas.com/um-menino-indiano-pre...,24/06/2020,NaN,0,o história ganhar fama rede social e diverso s...
3,Tuíte engana afirmar sol mata coronavírus,https://projetocomprova.com.br/publicações/tui...,2020/09/10,NaN,0,tuíte referência o estimativo estudar basear m...
4,Vietnã isola cidade registrou surto covid após...,https://noticias.uol.com.br/ultimas-noticias/e...,28/07/2020 18h46,NaN,1,O Vietnã suspendeu a partir de hoje todas as r...
...,...,...,...,...,...,...
4764,Marcos Pontes voluntário testes medicamento co...,https://noticias.uol.com.br/politica/ultimas-n...,31/07/2020 11h21,NaN,1,o ministrar ciência tecnologia e inovação marc...
4765,feirar chinês vender rato morcego origem coron...,https://www.e-farsas.com/feira-chinesa-vende-r...,29/01/2020,NaN,0,o vídeo minuto duração e espalhar rapidamente ...
4766,Não evidências científicas sobre necessidade t...,https://checamos.afp.com//nao-ha-evidencias-ci...,4/06/2021 às 22:25,NaN,0,“ trocar escovar covid-19 precisar trocar esco...
4767,Coronavírus atual pode versão contagiosa origi...,https://www.uol.com.br/vivabem/noticias/afp/20...,02/07/2020 21h06Atualizada em 03/07/2020 09h02,NaN,1,o variante sars-cov-2 dominante infecto célula...


In [22]:
# Cria dataframe para processamento
df = dataframe.copy()
df_sorted = df.sort_values(by='noticia_falsa').reset_index(drop=True)

# Transformar a coluna título em string
df_sorted['title'] = df_sorted['title'].astype(str)

In [23]:
### Filtrando notícias com valores 0 e 1
noticias_falsa_0 = df_sorted[df_sorted['noticia_falsa'] == 0]
noticias_falsa_1 = df_sorted[df_sorted['noticia_falsa'] == 1] # Aqui estão representadas as notícias verdadeiras

### Selecionar, aleatoriamente, 1000 notícias com valores 0 e 1500 com valores 1
amostra_0 = noticias_falsa_0.sample(n=1000, random_state=42)
amostra_1 = noticias_falsa_1.sample(n=1500, random_state=42)

# Criando um novo dataframe com estas notícias
novo_df = pd.concat([amostra_0, amostra_1])

# Resetando o index
novo_df = novo_df.sample(frac=1, random_state=24).reset_index(drop=True)

display(novo_df)

,title,link,date,Site,noticia_falsa,corpo_texto
0,Ministro britânico Saúde nega ter mentido sobr...,https://noticias.uol.com.br/ultimas-noticias/a...,27/05/2021 09h01,NaN,1,o ministrar britânico saudar matt hancock nega...
1,Covid-19: entidade orienta sobre vacinação pac...,https://www.uol.com.br/vivabem/noticias/redaca...,17/01/2021 10h22,NaN,1,o sbr sociedade brasileiro reumatologia lançar...
2,Exército construiu 2.000 leitos hospital apena...,https://www.boatos.org/saude/exercito-construi...,05/04/2020,NaN,0,chino 1.000 leito dia x exército brasil 2.000 ...
3,Vacina indiana contra covid: alvo rede privada...,https://www.uol.com.br/vivabem/noticias/bbc/20...,04/01/2021 08h31,NaN,1,Especialistas em doenças infecciosas estão man...
4,nasce diagnóstico psiquiátrico?,https://medicospelavidacovid19.com.br/opiniao/...,Data não disponível,Médicos Pela Vida,0,"['nasce', 'diagnóstico', 'psiquiátrico', 'skip..."
...,...,...,...,...,...,...
2495,Infectologista prevê variante Manaus predomina...,https://noticias.uol.com.br/saude/ultimas-noti...,29/01/2021 10h37,NaN,1,"O infectologista Marcus Lacerda, da Fiocruz-AM..."
2496,Dilma disse vacina chinesa vai funcionar porqu...,https://projetocomprova.com.br/publicações/dil...,2020/10/29,NaN,0,o postagem verificar inventar declaração o efi...
2497,Especialistas pedem atenção saúde mental confi...,https://www.uol.com.br/vivabem/noticias/afp/20...,19/04/2020 11h45,NaN,1,consequência psíquico confinamento afeta quase...
2498,É falso nova lei acabe obrigatoriedade uso más...,https://noticias.uol.com.br/confere/ultimas-no...,27/10/2020 04h00,NaN,0,imagem circular whatsapp espalhar mentira o le...


In [24]:
# Agora, separaremos uma porção para validação com dados externos

### Separar as notícias restantes
restantes_0 = noticias_falsa_0.drop(amostra_0.index)
restantes_1 = noticias_falsa_1.drop(amostra_1.index)

# Selecionar 449 notícias restantes com valor 0 e 800 com valor 1
amostra_0_restante = restantes_0.sample(n=449, random_state=42)
amostra_1_restante = restantes_1.sample(n=800, random_state=42)

# Criar um novo dataframe
segundo_df = pd.concat([amostra_0_restante, amostra_1_restante])

# Bagunçar o df
segundo_df = segundo_df.sample(frac=1, random_state=48).reset_index(drop=True)

display(segundo_df)

,title,link,date,Site,noticia_falsa,corpo_texto
0,Presidente Pfizer recusa suspender patentes va...,https://noticias.uol.com.br/ultimas-noticias/a...,06/05/2021 14h12Atualizada em 06/05/2021 18h59,NaN,1,o presidente pfizer albert bourla n quinta-fei...
1,Caixa abre duas horas cedo partir segunda evit...,https://economia.uol.com.br/noticias/redacao/2...,03/05/2020 14h19,NaN,1,o d segunda-feira o caixa antecipar horar o ab...
2,Teich: 'Brasil países melhor performa relação ...,https://noticias.uol.com.br/saude/ultimas-noti...,22/04/2020 18h09Atualizada em 22/04/2020 19h45,NaN,1,o ministrar saudar nelson teich hoje coletiva ...
3,defendeu,https://www.cnnbrasil.com.br/saude/exigencia-d...,2024-09-03T14:22:41-03:00,Médicos Pela Vida,0,"['exigência', 'passaporte', 'vacinal', 'respei..."
4,vacinação,https://www.gov.br/saude/pt-br/vacinacao,Data não disponível,Gov - Fake,0,"['vacinação', 'ministério', 'saúde', 'ir', 'co..."
...,...,...,...,...,...,...
1244,CoronaVac deve ter eficácia similar adultos id...,https://www.uol.com.br/vivabem/reuters/2020/12...,29/12/2020 18h48,NaN,1,"A CoronaVac, vacina contra Covid-19 do laborat..."
1245,"Brasil pode ganhar R$ 2,8 trilhões 'economia v...",https://economia.uol.com.br/noticias/estadao-c...,13/08/2020 07h50,NaN,1,o movimentar recuperação economia o abalar pro...
1246,Coronavírus: uso máscaras comércio passa obrig...,https://noticias.uol.com.br/saude/ultimas-noti...,11/05/2020 08h48Atualizada em 11/05/2020 08h53,NaN,1,tornar obrigatório o máscara transportar públi...
1247,"porque nós, mpv, criamos aba ‘fact checking’",https://medicospelavidacovid19.com.br/editoria...,Data não disponível,Médicos Pela Vida,0,"['porque', 'mpv', 'criamos', 'aba', 'fact', 'c..."


In [26]:
# Variáveis para padronização/configuração

scores = ['accuracy', 'precision', 'roc_auc']
usar_links = False

In [27]:
# Checar se há títulos vazios
non_empty_titles = novo_df['title'].apply(lambda x: len(x.strip()) > 0)
non_empty_links = novo_df['link'].apply(lambda x: len(x.strip()) > 0)

# Checar o resultado
print(non_empty_titles.head())
print("Há documentos vazios?", not non_empty_titles.all())
print("Há links vazios?", not non_empty_links.all())
print("Número de documentos vazios:", (~non_empty_titles).sum())
print("Número de links vazios:", (~non_empty_links).sum())

0    True
1    True
2    True
3    True
4    True
Name: title, dtype: bool
Há documentos vazios? False
Há links vazios? False
Número de documentos vazios: 0
Número de links vazios: 0


In [28]:
# Iniciar o processo de TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Nesta célula, será aplicado o TF-IDF aos títulos das notícias, caso não haja nenhum vazio
temp_tit = novo_df['title'].tolist()
if non_empty_titles.all():
    titles_tfidf = tfidf_vectorizer.fit_transform(temp_tit)
    print('TF-IDF aplicado com sucesso.')
    tfidf = titles_tfidf
else:
    print('Há documentos vazios ou apenas stop words.')

TF-IDF aplicado com sucesso.


In [29]:
def split_and_clean(link):
# Dividir o link por '/', '-', '%', '.', e ':'
    words = re.split(r'[/\-\%\.\:\_]', link)
    # Remover palavras indesejadas, stopwords e letras isoladas, mantendo acentos
    unwanted = {'https', 'www', 'com', 'br', 'pt', 'html', 'htm', 'ghtml', 'php'}
    cleaned_words = [
    word for word in words
    if not any(unwanted_word in word.lower() for unwanted_word in unwanted)
    and len(word) > 1
    ]
    return cleaned_words

if usar_links:

    # Tratando os links
    df['processed_links'] = df['link'].apply(split_and_clean)

    # TF-IDF para os combinação título e link
    temp_link = [' '.join(i) for i in df['processed_links']]
    temp_tit_link = [' '.join(list(a)) for a in zip(temp_tit, temp_link)]
    print(temp_tit_link)
    if non_empty_links.all():
        tit_link_tfidf = tfidf_vectorizer.fit_transform(temp_tit_link)
        print('TF-IDF aplicado com sucesso.')
        tfidf = tit_link_tfidf
    else:
        print('Há items vazios ou apenas stop words.')

In [30]:
# Criação de vetores
X = tfidf
y = novo_df['noticia_falsa']

titulos = novo_df['title'].tolist()

print("Dimensões de X:", X.shape)
print("Dimensões de y:", y.shape)

Dimensões de X: (2500, 5889)
Dimensões de y: (2500,)


## Modelos de Classificação
### Árvore de Decisão

In [31]:
# Padronizando scores para todos os modelos.

scores = ['accuracy', 'precision', 'roc_auc']

In [32]:
# Cria modelo
modelo_ad = DecisionTreeClassifier()
modelo_ad

# Validação cruzada
resultados_ad = cross_validate(modelo_ad, X, y, scoring=scores)

acc_ad = resultados_ad['test_accuracy'].mean()
prec_ad = resultados_ad['test_precision'].mean()
rocauc_ad = resultados_ad['test_roc_auc'].mean()
print(f'Acurácia: {acc_ad:.3f}\nPrecisão: {prec_ad:.3f}\nROC-AUC: {rocauc_ad:.3f}')

Acurácia: 0.834
Precisão: 0.867
ROC-AUC: 0.829


### Floresta Aleatória

In [33]:
# Cria modelo
modelo_rf = RandomForestClassifier()
modelo_rf

# Validação cruzada
resultados_rf = cross_validate(modelo_rf, X, y, scoring=scores)

acc_rf = resultados_rf['test_accuracy'].mean()
prec_rf = resultados_rf['test_precision'].mean()
rocauc_rf = resultados_rf['test_roc_auc'].mean()
print(f'Acurácia: {acc_rf:.3f}\nPrecisão: {prec_rf:.3f}\nROC-AUC: {rocauc_rf:.3f}')

Acurácia: 0.859
Precisão: 0.883
ROC-AUC: 0.921


### Modelo Regressão Logística

In [34]:
# Cria modelo
modelo_lr = LogisticRegression()
modelo_lr

# Validação cruzada
resultados_lr = cross_validate(modelo_lr, X, y, scoring=scores)

acc_lr = resultados_lr['test_accuracy'].mean()
prec_lr = resultados_lr['test_precision'].mean()
rocauc_lr = resultados_lr['test_roc_auc'].mean()
print(f'Acurácia: {acc_lr:.3f}\nPrecisão: {prec_lr:.3f}\nROC-AUC: {rocauc_lr:.3f}')

Acurácia: 0.843
Precisão: 0.799
ROC-AUC: 0.950


### SVC

In [35]:
# Parâmetros
C = 77
kernel = 'rbf'
gamma = 0.01

# Cria modelo
modelo_svm = svm.SVC(C=C, kernel=kernel, gamma=gamma)

# Validação cruzada
resultados_svm = cross_validate(modelo_svm, X, y, scoring=scores)

acc_svm = resultados_svm['test_accuracy'].mean()
prec_svm = resultados_svm['test_precision'].mean()
rocauc_svm = resultados_svm['test_roc_auc'].mean()
print(f'Acurácia: {acc_svm:.3f}\nPrecisão: {prec_svm:.3f}\nROC-AUC: {rocauc_svm:.3f}')

Acurácia: 0.886
Precisão: 0.870
ROC-AUC: 0.959


## Predição teste com dados externos

In [36]:
df_externo = pd.read_csv('../Notícias Fake e Verdadeiras.txt', header=None, sep='\t', on_bad_lines='skip')
display(df_externo)

,0,1
0,Vacinas contra a Covid-19 são mais perigosas q...,0
1,Vacina tem como objetivo matar seres humanos,0
2,Profissionais de saúde morrem por ataque cardí...,0
3,Butantan não confirma eficácia da CoronaVac em...,0
4,Vacinas de RNA mensageiro vão provocar morte e...,0
5,A vacina contra a Covid-19 vai modificar o DNA...,0
6,A vacina contra a Covid-19 tem chip líquido e ...,0
7,Imunizantes contra Covid-19 estão relacionados...,0
8,Vacinas contra Covid-19 criam campo magnético ...,0
9,CoronaVac não tem comprovação científica,0


In [37]:
stop_words = set(stopwords.words('portuguese'))

def trata_padrao(titulo, usar_links=False, link=None):
    '''
    Função para padronizar o tratamento dos títulos.

    Args:
    titulo (str) = título da notícia
    usar_links (bool) = usar ou não o link
    link (str) = link da notícia
    '''

    # Pré-processamento do titulo

    titulo = str(titulo).lower()
    titulo = nltk.word_tokenize(titulo)
    titulo = [word for word in titulo if word not in stop_words]
    titulo = [' '.join(titulo)]
    temp_tit = titulo

    # se usar link, pre processar link
    if usar_links:
        link = [split_and_clean(link)]
        temp_link = [' '.join(i) for i in link]
        print(temp_tit,temp_link)
        temp_tit_link = [' '.join(list(a)) for a in zip(temp_tit, temp_link)]
        processed = temp_tit_link
    else:
        processed = titulo

    return processed

In [38]:
# Define modelos para teste
modelos = [modelo_ad, modelo_rf, modelo_lr, modelo_svm]

# Treina modelos com todos os dados
for modelo in modelos:
    modelo.fit(X, y)

# Função para predizer novos títulos
def predict_title(titulo):
    preprocessed_title = trata_padrao(titulo)

    tfidf_vector = tfidf_vectorizer.transform(preprocessed_title)
    preds = dict()

    for modelo in modelos:
        preds[modelo] = modelo.predict(tfidf_vector)

    return preds

In [39]:
def comparar(df):
    '''
    Função para comparar predições de diferentes modelos com o target

    Args:
        df (pandas.df): dataframe com coluna '1' sendo os títulos/corpus, e '2' com o target/label
        
    Returns:
        list: Uma lista contendo 4 listas (uma para cada modelo: Decision Tree, Random Forest, Logistic Regression, SVC)
    '''
    DT, RF, LR, SVC = [], [], [], []
    for index, row in df.iterrows():
        text = row[0]
        target = row[1]
        pred = predict_title(text)
        pred['Target'] = [target]
        
        # Guardar as previsões em suas respectivas listas
        keys = [i for i in pred.keys()]
        DT.append(pred[keys[0]][0])
        RF.append(pred[keys[1]][0])
        LR.append(pred[keys[2]][0])
        SVC.append(pred[keys[3]][0])
        
        print(f'Notícia: {text}\n')
        for i in pred:
            print("\t{}\t{}".format(i, pred[i]))
        print('-----------------------------------------------------')
        print()
        
    return [DT, RF, LR, SVC]
        

# Prediz df externo    
predictions = comparar(df_externo)

Notícia: Vacinas contra a Covid-19 são mais perigosas que o próprio vírus

	DecisionTreeClassifier()	[0]
	RandomForestClassifier()	[0]
	LogisticRegression()	[0]
	SVC(C=77, gamma=0.01)	[0]
	Target	[0]
-----------------------------------------------------

Notícia: Vacina tem como objetivo matar seres humanos

	DecisionTreeClassifier()	[0]
	RandomForestClassifier()	[0]
	LogisticRegression()	[0]
	SVC(C=77, gamma=0.01)	[0]
	Target	[0]
-----------------------------------------------------

Notícia: Profissionais de saúde morrem por ataque cardíaco em decorrência da vacina

	DecisionTreeClassifier()	[0]
	RandomForestClassifier()	[1]
	LogisticRegression()	[1]
	SVC(C=77, gamma=0.01)	[1]
	Target	[0]
-----------------------------------------------------

Notícia: Butantan não confirma eficácia da CoronaVac em idosos

	DecisionTreeClassifier()	[0]
	RandomForestClassifier()	[1]
	LogisticRegression()	[1]
	SVC(C=77, gamma=0.01)	[1]
	Target	[0]
-----------------------------------------------------

N

In [40]:
# Calcula score de cada método para df externo
target = df_externo[1].tolist()

acc_dt_e = accuracy_score(target, predictions[0])
acc_rf_e = accuracy_score(target, predictions[1])
acc_lr_e = accuracy_score(target, predictions[2])
acc_svc_e = accuracy_score(target, predictions[3])

prec_dt_e = precision_score(target, predictions[0])
prec_rf_e = precision_score(target, predictions[1])
prec_lr_e = precision_score(target, predictions[2])
prec_svc_e = precision_score(target, predictions[3])

print('---- Decision Tree ----')
print(f'Acurácia: {acc_dt_e:.3f}\nPrecisão: {prec_dt_e:.3f}')
print()
print('---- Random Forest ----')
print(f'Acurácia: {acc_rf_e:.3f}\nPrecisão: {prec_rf_e:.3f}')
print()
print('---- Linear Regression ----')
print(f'Acurácia: {acc_lr_e:.3f}\nPrecisão: {prec_lr_e:.3f}')
print()
print('---- SVC ----')
print(f'Acurácia: {acc_svc_e:.3f}\nPrecisão: {prec_svc_e:.3f}')
print()

---- Decision Tree ----
Acurácia: 0.700
Precisão: 0.667

---- Random Forest ----
Acurácia: 0.650
Precisão: 0.600

---- Linear Regression ----
Acurácia: 0.700
Precisão: 0.643

---- SVC ----
Acurácia: 0.700
Precisão: 0.643



### Testando com outro Dataset

In [41]:
df_titulos = pd.read_csv('df_titulos.csv', header=None, sep=',', on_bad_lines='skip')

display(df_titulos)

,0,1
0,0,1.0
1,Por que é mais difícil para as mulheres lutar ...,1.0
2,Cientista modifica forma de bactérias para com...,1.0
3,Qual o melhor tipo de cirurgia para miopia?,1.0
4,Psicólogo cria 'kit de reparo' para descontent...,1.0
...,...,...
1308,Anvisa aprova novo medicamento para tratamento...,1.0
1309,Artigo contrário à relação HIV-Aids é 'despubl...,1.0
1310,Mulheres buscam redução da bochecha para afina...,1.0
1311,"Temos de tratar da vida sexual dos gordos, def...",1.0


In [42]:
# Prediz um segundo DataFrame externo, contendo mais notícias, sendo estas verdadeiras

predictions = comparar(df_titulos)

Notícia: 0

	DecisionTreeClassifier()	[0]
	RandomForestClassifier()	[0]
	LogisticRegression()	[1]
	SVC(C=77, gamma=0.01)	[1]
	Target	[1.0]
-----------------------------------------------------

Notícia: Por que é mais difícil para as mulheres lutar contra alcoolismo e dependência às drogas

	DecisionTreeClassifier()	[1]
	RandomForestClassifier()	[1]
	LogisticRegression()	[1]
	SVC(C=77, gamma=0.01)	[1]
	Target	[1.0]
-----------------------------------------------------

Notícia: Cientista modifica forma de bactérias para combatê-las

	DecisionTreeClassifier()	[0]
	RandomForestClassifier()	[0]
	LogisticRegression()	[1]
	SVC(C=77, gamma=0.01)	[0]
	Target	[1.0]
-----------------------------------------------------

Notícia: Qual o melhor tipo de cirurgia para miopia?

	DecisionTreeClassifier()	[0]
	RandomForestClassifier()	[0]
	LogisticRegression()	[1]
	SVC(C=77, gamma=0.01)	[0]
	Target	[1.0]
-----------------------------------------------------

Notícia: Psicólogo cria 'kit de reparo' par

In [43]:
# Calcula score de cada método para df externo
target = df_titulos[1].tolist()

acc_dt_e = accuracy_score(target, predictions[0])
acc_rf_e = accuracy_score(target, predictions[1])
acc_lr_e = accuracy_score(target, predictions[2])
acc_svc_e = accuracy_score(target, predictions[3])

prec_dt_e = precision_score(target, predictions[0])
prec_rf_e = precision_score(target, predictions[1])
prec_lr_e = precision_score(target, predictions[2])
prec_svc_e = precision_score(target, predictions[3])

print('---- Decision Tree ----')
print(f'Acurácia: {acc_dt_e:.3f}\nPrecisão: {prec_dt_e:.3f}')
print()
print('---- Random Forest ----')
print(f'Acurácia: {acc_rf_e:.3f}\nPrecisão: {prec_rf_e:.3f}')
print()
print('---- Linear Regression ----')
print(f'Acurácia: {acc_lr_e:.3f}\nPrecisão: {prec_lr_e:.3f}')
print()
print('---- SVC ----')
print(f'Acurácia: {acc_svc_e:.3f}\nPrecisão: {prec_svc_e:.3f}')
print()

---- Decision Tree ----
Acurácia: 0.478
Precisão: 1.000

---- Random Forest ----
Acurácia: 0.423
Precisão: 1.000

---- Linear Regression ----
Acurácia: 0.945
Precisão: 1.000

---- SVC ----
Acurácia: 0.816
Precisão: 1.000



**Obs: A precisão é igual a 1 em todos devido a todas as notícias serem verdadeiras, o que impacta a relação: $ \frac{vp}{vp + fp} $, uma vez que todos são positivos.** 

### Teste com dados que foram separados para validação

In [44]:
segundo_df = segundo_df[['title', 'noticia_falsa']]

#segundo_df.columns = ['', '']

display(segundo_df)

,title,noticia_falsa
0,Presidente Pfizer recusa suspender patentes va...,1
1,Caixa abre duas horas cedo partir segunda evit...,1
2,Teich: 'Brasil países melhor performa relação ...,1
3,defendeu,0
4,vacinação,0
...,...,...
1244,CoronaVac deve ter eficácia similar adultos id...,1
1245,"Brasil pode ganhar R$ 2,8 trilhões 'economia v...",1
1246,Coronavírus: uso máscaras comércio passa obrig...,1
1247,"porque nós, mpv, criamos aba ‘fact checking’",0


In [45]:
predictions = comparar(segundo_df)

Notícia: Presidente Pfizer recusa suspender patentes vacinas contra covid

	DecisionTreeClassifier()	[1]
	RandomForestClassifier()	[1]
	LogisticRegression()	[1]
	SVC(C=77, gamma=0.01)	[1]
	Target	[1]
-----------------------------------------------------

Notícia: Caixa abre duas horas cedo partir segunda evitar aglomeração

	DecisionTreeClassifier()	[1]
	RandomForestClassifier()	[1]
	LogisticRegression()	[1]
	SVC(C=77, gamma=0.01)	[1]
	Target	[1]
-----------------------------------------------------

Notícia: Teich: 'Brasil países melhor performa relação covid'

	DecisionTreeClassifier()	[1]
	RandomForestClassifier()	[1]
	LogisticRegression()	[1]
	SVC(C=77, gamma=0.01)	[1]
	Target	[1]
-----------------------------------------------------

Notícia: defendeu

	DecisionTreeClassifier()	[0]
	RandomForestClassifier()	[0]
	LogisticRegression()	[1]
	SVC(C=77, gamma=0.01)	[1]
	Target	[0]
-----------------------------------------------------

Notícia: vacinação

	DecisionTreeClassifier()	[0]
	R

In [46]:
# Calcula score de cada método para df externo
target = segundo_df['noticia_falsa'].tolist()

acc_dt_e = accuracy_score(target, predictions[0])
acc_rf_e = accuracy_score(target, predictions[1])
acc_lr_e = accuracy_score(target, predictions[2])
acc_svc_e = accuracy_score(target, predictions[3])

prec_dt_e = precision_score(target, predictions[0])
prec_rf_e = precision_score(target, predictions[1])
prec_lr_e = precision_score(target, predictions[2])
prec_svc_e = precision_score(target, predictions[3])

print('---- Decision Tree ----')
print(f'Acurácia: {acc_dt_e:.3f}\nPrecisão: {prec_dt_e:.3f}')
print()
print('---- Random Forest ----')
print(f'Acurácia: {acc_rf_e:.3f}\nPrecisão: {prec_rf_e:.3f}')
print()
print('---- Linear Regression ----')
print(f'Acurácia: {acc_lr_e:.3f}\nPrecisão: {prec_lr_e:.3f}')
print()
print('---- SVC ----')
print(f'Acurácia: {acc_svc_e:.3f}\nPrecisão: {prec_svc_e:.3f}')
print()

---- Decision Tree ----
Acurácia: 0.834
Precisão: 0.897

---- Random Forest ----
Acurácia: 0.852
Precisão: 0.907

---- Linear Regression ----
Acurácia: 0.888
Precisão: 0.863

---- SVC ----
Acurácia: 0.901
Precisão: 0.905

